## TP 01 - Extraction des features

In [1]:
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import cv2
import numpy as np
import time

pathToData = 'C:/Users/aroma/Documents/GTI770/data/'

# Données de l'exercice
df = pd.read_csv(pathToData + 'csv/galaxy/galaxy_label_data_set.csv')

### Partie 1 - Seuils de binarisation
On va tenter dans un premier temps d'écrire un programme qui prendra une image et un seuil et qui construira l'image binaire correspondante

In [2]:
# frac_img_bi permet de ressortir la fraction du nombre de pixel blanc dans une image binarisé pour un seuil donné
def frac_img_bi(img, seuil):
    compteur = 0
    for i in range (img.shape[0]) :
        for j in range (img.shape[1]) : 
#On calcule le niveau de gris de chaque pixel et on applique le seuil pour rendre l'image binaire
            if int(0.2989*img[i][j][0] +
                   0.5870*img[i][j][1] +
                   0.1140*img[i][j][2]) >seuil :
               compteur+=1
    return compteur/(img.shape[0]*img.shape[1])

# extract_binar permet de ressortir la liste des features pour une liste d'image et une liste de seuil donnés
def extract_binar (image) :
    list_seuil =  [68,128,192]
    list_binaire = []
    for k in list_seuil :
        list_binaire.append(frac_img_bi(image, k))
    return list_binaire

### Partie 2 : Couleur du pixel central
D'après la littérature, la couleur de la galaxie est une caractéristique discriminatoire pour différencier les galaxies smooth et spiral. Nous allons seulement récupérer les informations du pixel central pour ne pas nous encombrer de features trop volumineuses.

In [3]:
def pixel_central (img) : 
    return img[int(img.shape[0]/2)][int(img.shape[1]/2)].tolist()

### Partie 3 : Détection de contour
En étudiant plusieurs photos disponibles on remarque que les galaxies sont de tailles différentes, on souhaite donc étudier leur périmètre. Pour cela on a décidé de mettre en place un algorithme de détection de contour. Les pixels concernés auront alors une couleur blanche. Nous prendrons le nombre de pixel blanc par image comme feature.

In [4]:
def det_contour (idImg) :
    img = cv2.imread(pathToData + 'images/' + idImg +'.jpg',0)
    edge = cv2.Canny(img,1,45)
    compteur = 0
    for k in range (edge.shape[0]) :
        for l in range (edge.shape[1]) :
            if edge[k][l] != 0 :
                compteur+=1
    return compteur/(edge.shape[0]*edge.shape[1])

### Partie 4 : Extraction des features
Nous avons définis toutes les fonctions pour les features et nous allons maintenant les extraire. On souhaite avoir un fichier de la forme suivante : Id de l'image, fractions après les différentes valeurs de seuil, pixel central et fraction sur les contours.

In [5]:
#On instancie différentes variable pour stocker les futures liste de features et target
list_img = []
# k est le compteur pour gérer le nombre d'image qu'on veut traiter
k = 0

#On recherche 20 valeurs
while k<10:
    img = io.imread(pathToData + 'images/' + str(df.iloc[k][0]) + '.jpg')

    #Coord new origin point
    xcoord = int(img.shape[0]/2)-40
    ycoord = int(img.shape[1]/2)-40
    
    newimg = np.zeros((80,80,3), dtype=np.uint8)
    
    for i in range(80) :
        for j in range(80) : 
            newimg[i][j] = img[ycoord+i][xcoord+j]
    list_img.append([df.iloc[k][0],cv2.fastNlMeansDenoisingColored(newimg,None,10,10,7,21), df.iloc[k][1]])
    k+=1   

In [6]:
def processImg(image):
    res_binaire  = extract_binar(image[1])
    res_pixel = pixel_central(image[1])
    res_contour = det_contour(str(image[0]))
    resultats.append([image[0],res_binaire[0],res_binaire[1],res_binaire[2],res_pixel[0],
                      res_pixel[1],res_pixel[2],res_contour, image[2]])

In [7]:
resultats = []

start_time = time.time()
with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = [executor.submit(processImg, image) for image in list_img]
print("Temps d execution : %s secondes ---" % (time.time() - start_time))

Temps d execution : 11.654747486114502 secondes ---


In [9]:
extract = pd.DataFrame(resultats)
extract.columns = ['image name','threshold value 1','threshold value 2','threshold value 3',
                   'central pixel red','central pixel green','central pixel blue','contour detection','galaxy_type']
extract.to_csv('test')